##  1: Importing Libraries and Loading Data

**To do list:**

*   **Centralized Import**: At the very beginning of the Notebook, we import all the necessary Python libraries at once, including pandasfor data processing, numpyfor scientific computing, and various modules from scikit-learnfor feature engineering, model training, and evaluation. This is a good programming practice, making the code's dependencies clear at a glance.
*   **Data Loading**: We use pd.read_csv()to load the three core files provided by the competition: train.csv(training data), test.csv(test data), and sample_submission.csv(submission format example).

In [4]:
# Data Processing and Scientific Computing

import pandas as pd
import numpy as np

# Feature Engineering

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

# Models and Evaluation

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

# Deep Learning Embedding Models

from sentence_transformers import SentenceTransformer

print("🚀 Team 8 Baseline & Embedding Model Pipeline Started!")

# --- Load Dataset ---

# Use try-except structure to ensure robust data loading

try:
    train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
    test = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
    sample = pd.read_csv("/kaggle/input/llm-classification-finetuning/sample_submission.csv")
    print("✅ Dataset loaded successfully!")
    print(f"  Training set size: {train.shape}")
    print(f"  Test set size: {test.shape}")
except FileNotFoundError:
    print("❌ Data loading failed! Please check if the competition dataset has been correctly added to the Notebook.")

🚀 Team 8 Baseline & Embedding Model Pipeline Started!
✅ Dataset loaded successfully!
  Training set size: (57477, 9)
  Test set size: (3, 4)


## 2: Basic Feature Engineering 
**Core Principle**: To ensure the fairness and effectiveness of the model, all feature engineering performed on the training set `train` must be applied to the test set `test` in exactly the same way. This guarantees that the data structure seen by the model during training and prediction is completely consistent, thereby avoiding common errors such as `KeyError`. 

**1. Label Generation (`label` - Only for Training Set)**:
*   **Function**: Convert the three columns `winner_model_a`, `winner_model_b`, and `winner_tie` (in one-hot encoded format) in the training data into a single, multi-class label column.
*   **Method**: `.argmax(axis=1)` will search for the index of the maximum value along each row. For instance, if `winner_model_a` is 1, its index is 0; if `winner_model_b` is 1, the index is 1; and if it's a tie, the index is 2. Thus, we obtain a target variable `y` suitable for training a classification model (with values 0, 1, 2).

**2. Text and Length Feature Engineering (for `train` and `test` sets):**
*   **Text Concatenation (`text`)**: We concatenate `prompt` with each of the two `responses` (`response_a`, `response_b`) respectively, and then connect the two complete response texts using a special delimiter `[SEP]`. The `[SEP]` delimiter provides a clear boundary for the model, which helps it better distinguish and compare the two different responses.
*   **Length Features (`..._len`)**: We extract the character lengths of `prompt`, `response_a`, and `response_b` as additional numerical features. Text length itself is a useful signal in many NLP tasks.

In [5]:
# === Step 2: Feature Engineering (Consistent processing for train and test sets) ===

print("\n--- Executing Step 1: Feature Engineering ---")

# --- 1. Process training set (train) ---
# Convert one-hot encoded labels to single multi-class labels (0, 1, 2)
train['label'] = train[['winner_model_a', 'winner_model_b', 'winner_tie']].values.argmax(axis=1)

# Create complete text containing prompt and response
train['text_a'] = train['prompt'] + " " + train['response_a']
train['text_b'] = train['prompt'] + " " + train['response_b']

# Concatenate two responses with [SEP] separator for model comparison
train['text'] = train['text_a'] + " [SEP] " + train['text_b']

# Calculate length features
train['prompt_len'] = train['prompt'].str.len()
train['resp_a_len'] = train['response_a'].str.len()
train['resp_b_len'] = train['response_b'].str.len()


# --- 2. Process test set (test) with identical method ---
# Note: Test set doesn't have 'label' column, so no need to process
test['text_a'] = test['prompt'] + " " + test['response_a']
test['text_b'] = test['prompt'] + " " + test['response_b']
test['text'] = test['text_a'] + " [SEP] " + test['text_b']
test['prompt_len'] = test['prompt'].str.len()
test['resp_a_len'] = test['response_a'].str.len()
test['resp_b_len'] = test['response_b'].str.len()

print("✅ Feature engineering completed.")



--- Executing Step 1: Feature Engineering ---
✅ Feature engineering completed.


## 3: Baseline Model (Bag of Words + Logistic Regression) 

**1. Vectorization**:
*   **Text Vectorization (`CountVectorizer`)**: We use the "Bag of Words" model to convert text data into a numerical matrix. The `max_features=5000` parameter limits the use of only the 5000 most common words/phrases as features to control the dimensionality. The `ngram_range=(1,2)` parameter enables the model to consider both individual words (unigrams) and adjacent word pairs (bigrams) simultaneously, capturing richer phrase information.
*   **Standardization of Numerical Features (`StandardScaler`)**: We standardize the length features we created earlier to have a mean of 0 and a variance of 1. This step is crucial because models like logistic regression are sensitive to the scale of features, and standardization prevents the model from overly emphasizing features with a larger numerical range.
*   **Feature Concatenation (`np.hstack`)**: We horizontally concatenate the processed text feature matrix and the numerical feature matrix to form the final complete feature matrix `X_baseline` for training the baseline model.

**2. Training and Validation**:
*   **Data Splitting (`train_test_split`)**: We split the feature matrix and labels into training and validation sets in an 8:2 ratio. `random_state=42` ensures that the split results are consistent each time, guaranteeing the reproducibility of the experiment.
*   **Model Training (`LogisticRegression`)**: We choose logistic regression as the baseline classifier. It is a simple, fast, and interpretable linear model, making it an ideal starting point for the project. We set `max_iter=1000` to increase the maximum number of iterations, ensuring that the model is fully trained and converges, and avoiding `ConvergenceWarning`.
*   **Performance Evaluation**: After training the model on the training set, we make predictions on the unseen validation set and calculate the validation score using the official competition evaluation metric `log_loss`. This score helps us quickly assess the model's performance before submission.

In [ ]:
# === Step 3: Baseline Model (Bag of Words + Logistic Regression) ===

print("\n--- Executing Baseline Model ---")

# --- 1. Feature Vectorization ---
# Text features (Bag of Words), considering unigrams and bigrams
vectorizer = CountVectorizer(max_features=5000, ngram_range=(1,2))
X_text_train = vectorizer.fit_transform(train['text'])
X_text_test = vectorizer.transform(test['text'])

# Numerical features (lengths), with standardization
scaler = StandardScaler()
num_features_train = train[['prompt_len','resp_a_len','resp_b_len']]
num_features_test = test[['prompt_len','resp_a_len','resp_b_len']]
X_num_train = scaler.fit_transform(num_features_train)
X_num_test = scaler.transform(num_features_test)

# Combine text and numerical features
X_baseline = np.hstack([X_text_train.toarray(), X_num_train])
X_test_baseline = np.hstack([X_text_test.toarray(), X_num_test])
y = train['label']

# --- 2. Training and Validation ---
# Split baseline features into training and validation sets
X_train_base, X_val_base, y_train_base, y_val_base = train_test_split(
    X_baseline, y, test_size=0.2, random_state=42
)

# Train logistic regression model, increased max_iter to avoid convergence warning
clf_base = LogisticRegression(max_iter=1000)
clf_base.fit(X_train_base, y_train_base)

# Evaluate model performance on validation set
y_pred_val_base = clf_base.predict_proba(X_val_base)
validation_score_base = log_loss(y_val_base, y_pred_val_base)
print(f"📊 Validation LogLoss (Baseline): {validation_score_base:.5f}")

# Note: We won't generate submission.csv from this model, the final submission will be from stronger models.



--- Executing Baseline Model ---


## 4: Embedding Model (MiniLM) 

**1. Model Loading (`SentenceTransformer`)**:

*   **Function**: We load a pre-trained Transformer model `all-MiniLM-L6-v2`. Unlike bag-of-words models, it can convert an entire sentence into a fixed-length vector (i.e., "embedding" or Embedding) that contains rich semantic information.
*   **Offline Mode**: To meet the reproducibility requirements of the competition (which usually requires disabling the network), we load the model from the Kaggle dataset path (`/kaggle/input/...`) that was pre-added to the Notebook instead of downloading it from the network. The `device='cuda'` parameter specifies the use of GPU for computation, which can greatly accelerate the subsequent encoding process.

**2. Generate Sentence Vectors (`model.encode`)**:

*   **Function**: This is the core step. We input the text of each sample into the MiniLM model and obtain a fixed-dimensional vector (384 dimensions for MiniLM).
*   **Comparison**: The features here are no longer the occurrence frequency of words but vector representations of deep semantics. This usually leads to a significant performance improvement over bag-of-words models. The `batch_size=128` parameter enables batch processing on the GPU, further enhancing the encoding efficiency.
  
 **3. Training and Validation**:
 
*   **Data Splitting**: Similar to the Baseline model, we also split the generated sentence vectors `train_emb` into training and validation sets to ensure a fair performance comparison.
*   **Model Training**: We again use a logistic regression classifier, but this time the input features are high-quality sentence vectors.
*   **Performance Evaluation**: We calculate and print the `log_loss` score of the Embedding model on the validation set. By comparing this score with the validation score of the Baseline model, we can quantitatively assess the performance improvement brought by upgrading from the bag-of-words model to the embedding model. 
*   **4. Generate the final submission file**:
*   **Prediction**: Use the trained Embedding model to predict the sentence vectors `test_emb` of the test set.
*   **File generation**: Save the predicted probabilities as the `submission.csv` file. **The file name must be `submission.csv`** to meet the submission requirements of the Kaggle competition. This file will overwrite any existing file with the same name, ensuring that the final submission is the result of our stronger Embedding model.

In [ ]:
# === Step 4: Embedding Model (MiniLM + Logistic Regression) ===

print("\n--- Executing Embedding Model ---")

# --- 1. Load pre-trained SentenceTransformer model (offline mode) ---
# Make sure you've added 'sentence-transformers-all-minilm-l6-v2' dataset in Notebook's Input
model_path = '/kaggle/input/sentencetransformersallminilml6v2'

model = None
try:
    model = SentenceTransformer(model_path, device='cuda') # Use GPU acceleration
    print("✅ Embedding model loaded successfully!")
except Exception as e:
    print(f"❌ Failed to load embedding model: {e}")
    print("  Please ensure 'sentencetransformersallminilml6v2' dataset is added in the Input panel and path is correct.")


# Only proceed if model loaded successfully
if model is not None:
    # --- 2. Generate sentence embeddings ---
    # Create a new combined field for embedding for both train and test data
    train['combined_for_embedding'] = train['prompt'] + " " + train['response_a'] + " [SEP] " + train['response_b']
    test['combined_for_embedding'] = test['prompt'] + " " + test['response_a'] + " [SEP] " + test['response_b']
    
    print("⏳ Generating sentence embeddings for training set (this may take a few minutes)...")
    train_emb = model.encode(train['combined_for_embedding'].tolist(), show_progress_bar=True, batch_size=128)
    
    print("⏳ Generating sentence embeddings for test set...")
    test_emb = model.encode(test['combined_for_embedding'].tolist(), show_progress_bar=True, batch_size=128)
    print("✅ Sentence embeddings generation completed.")

    # --- 3. Training and Validation ---
    # Split embedding features into training and validation sets
    X_train_emb, X_val_emb, y_train_emb, y_val_emb = train_test_split(
        train_emb, y, test_size=0.2, random_state=42
    )

    # Train logistic regression classifier
    print("⏳ Training classifier for Embedding model...")
    clf_emb = LogisticRegression(max_iter=1000)
    clf_emb.fit(X_train_emb, y_train_emb)
    print("✅ Classifier training completed.")

    # Evaluate model performance on validation set
    y_pred_val_emb = clf_emb.predict_proba(X_val_emb)
    validation_score_emb = log_loss(y_val_emb, y_pred_val_emb)
    print(f"📊 Validation LogLoss (Embedding): {validation_score_emb:.5f}")

    # --- 4. Generate final Kaggle submission file ---
    # Use model trained on partial data for predictions
    # (Better practice would be retraining on full data, but using clf_emb directly for speed and simplicity)
    print("⏳ Generating final predictions for test set...")
    preds_final = clf_emb.predict_proba(test_emb)

    # Create submission DataFrame, ensure filename is "submission.csv"
    submission_final = pd.DataFrame(preds_final, columns=sample.columns[1:])
    submission_final.insert(0, "id", sample["id"])
    submission_final.to_csv("submission.csv", index=False)

    print("\n🎉 Final submission.csv has been generated! Ready to save and submit.")

| Model | Kaggle Public Score | Validation LogLoss | 
| :--- | :--- | :--- | 
| Baseline (Bag of Words + Logistic Regression) | `1.29503` | `1.28668` | 
| Embedding (MiniLM) | `1.08498` | `1.08496` | 

## 5: Model Extensions

After completing the baseline model (bag-of-words + logistic regression) and the embedding model (MiniLM + logistic regression), in this stage we further explored various directions for improving the model performance. The main types of extensions include: 

### 5.1 Multi-model Embedding and Integration (E5 Embedding + LightGBM + Ensemble with MiniLM + LR)

**Objective:**
Based on the existing MiniLM sentence vectors, introduce another pre-trained embedding model, E5-small-v2, and enhance the generalization ability through model integration. 

**Key Points for Implementation:**
- **E5 Vector Generation:** Use SentenceTransformer to load E5-small-v2 and generate 384-dimensional embeddings for the prompt + response text.
- **LightGBM Classification:** Train a LightGBM model on the E5 embeddings (300 trees, `num_leaves=64`) and obtain Validation LogLoss ≈ 1.0838.
- **Soft Voting Fusion (Ensemble):** Combine MiniLM + logistic regression with E5 + LightGBM using `VotingClassifier(voting="soft")`, resulting in final LogLoss ≈ 1.0767. 
**Effect:**
  
> The integrated model achieved a further reduction in LogLoss compared to the single Embedding model on the validation set, indicating that the E5 semantic space and MiniLM are complementary to each other.


In [ ]:
# === Step 5: Model Extensions (E5 Embedding + LightGBM + Ensemble) ===
print("\n--- Executing Step 3: Model Extensions ---")

from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss
from sentence_transformers import SentenceTransformer

# --- 1. Load alternative embedding model (E5) ---
try:
    e5_path = "/kaggle/input/e5-small-v2"  # Ensure this is added to Input
    e5_model = SentenceTransformer(e5_path, device='cuda')
    print("✅ E5 model loaded successfully!")
except Exception as e:
    print("❌ Failed to load E5 model:", e)
    e5_model = None

if e5_model is not None:
    print("⏳ Generating E5 sentence embeddings...")
    train_emb_e5 = e5_model.encode(train["combined_for_embedding"].tolist(), batch_size=128, show_progress_bar=True)
    test_emb_e5 = e5_model.encode(test["combined_for_embedding"].tolist(), batch_size=128, show_progress_bar=True)
    print("✅ E5 embeddings generation completed.")

    # --- 2. LightGBM Classifier ---
    print("⏳ Training LightGBM model...")
    lgbm = LGBMClassifier(n_estimators=300, learning_rate=0.05, num_leaves=64, random_state=42)
    X_train_lgb, X_val_lgb, y_train_lgb, y_val_lgb = train_test_split(train_emb_e5, y, test_size=0.2, random_state=42)
    lgbm.fit(X_train_lgb, y_train_lgb)
    val_pred_lgb = lgbm.predict_proba(X_val_lgb)
    val_logloss_lgb = log_loss(y_val_lgb, val_pred_lgb)
    print(f"📊 Validation LogLoss (E5 + LightGBM): {val_logloss_lgb:.5f}")

    # --- 3. Ensemble with Logistic Regression (MiniLM) ---
    print("⏳ Performing ensemble fusion...")
    ensemble = VotingClassifier(
        estimators=[
            ('minilm', clf_emb),
            ('lgbm', lgbm)
        ],
        voting='soft'
    )
    X_train_ens, X_val_ens, y_train_ens, y_val_ens = train_test_split(
        np.hstack([train_emb, train_emb_e5]), y, test_size=0.2, random_state=42
    )
    ensemble.fit(X_train_ens, y_train_ens)
    y_pred_val_ens = ensemble.predict_proba(X_val_ens)
    val_logloss_ens = log_loss(y_val_ens, y_pred_val_ens)
    print(f"🎯 Validation LogLoss (MiniLM+E5 Ensemble): {val_logloss_ens:.5f}")

    # --- 4. Final Prediction and File Output ---
    preds_final_ens = ensemble.predict_proba(np.hstack([test_emb, test_emb_e5]))
    submission_final_ens = pd.DataFrame(preds_final_ens, columns=sample.columns[1:])
    submission_final_ens.insert(0, "id", sample["id"])
    submission_final_ens.to_csv("submission.csv", index=False)
    print("✅ Ensemble model results saved as submission.csv")


### 5.2 Bias-aware Modeling & Bias Analysis (Analysis of Bias Characteristics and Fairness) 

**Objective:**

Evaluate the model's biases in non-semantic factors such as position and redundancy, and attempt to construct "bias-aware" features. 

**Key Points for Implementation:**
- **Constructing Bias Features:**
  - Response length difference `len_diff`, length ratio `len_ratio`
  - Lexical diversity difference `lexical_diff`
- **Bias-aware Modeling:** Concatenate the MiniLM embedding after PCA dimensionality reduction with the above bias features and train with LightGBM to obtain Validation LogLoss ≈ 1.03038. The 4.32% improvement in LogLoss (1.07689 → 1.03038) demonstrates that explicitly modeling positional bias features alongside semantic embeddings enables the model to make more content-based decisions rather than relying on superficial characteristics.
- **Position Bias Experiment:**
  Randomly select 1,000 samples, swap the A/B responses and re-predict to obtain
  - Conversion rate ≈ 0.19
  - Average probability change ≈ 0.0168
  → The model still has a slight position bias, but overall it focuses on semantic content.

In [ ]:
# === Test Set Bias Feature Calculation ===
print("\n--- Executing: Test Set Bias Feature Calculation ---")

for df in [train, test]:
    # Calculate response lengths
    df["resp_a_len"] = df["response_a"].str.len()
    df["resp_b_len"] = df["response_b"].str.len()
    
    # Length difference features
    df["len_diff"] = df["resp_a_len"] - df["resp_b_len"]
    df["len_ratio"] = df["resp_a_len"] / (df["resp_b_len"] + 1e-6)  # Add small epsilon to avoid division by zero
    
    # Lexical diversity features (unique words / total words)
    df["lexical_a"] = df["response_a"].apply(lambda x: len(set(str(x).split())) / (len(str(x).split()) + 1e-6))
    df["lexical_b"] = df["response_b"].apply(lambda x: len(set(str(x).split())) / (len(str(x).split()) + 1e-6))
    df["lexical_diff"] = df["lexical_a"] - df["lexical_b"]

# Display sample features
print(train[["len_diff", "len_ratio", "lexical_diff"]].head())
print("✅ Test set bias feature calculation completed.")

In [ ]:
from sklearn.decomposition import PCA
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss
import numpy as np

print("\n--- Quick Bias Modeling Experiment ---")

# Combine bias features (MiniLM embedding + 3 bias features)
bias_feats_train = train[["len_diff", "len_ratio", "lexical_diff"]].fillna(0).values
bias_feats_test = test[["len_diff", "len_ratio", "lexical_diff"]].fillna(0).values

# 1️⃣ PCA Dimensionality Reduction
pca = PCA(n_components=128, random_state=42)
train_pca = pca.fit_transform(train_emb)
test_pca = pca.transform(test_emb)

# 2️⃣ Concatenate Bias Features
X_train_bias = np.hstack([train_pca, bias_feats_train])
X_test_bias = np.hstack([test_pca, bias_feats_test])

# 3️⃣ Train Fast LightGBM
lgb_bias = LGBMClassifier(
    n_estimators=200, learning_rate=0.05, num_leaves=64, random_state=42
)
X_train_b, X_val_b, y_train_b, y_val_b = train_test_split(
    X_train_bias, y, test_size=0.2, random_state=42
)
lgb_bias.fit(X_train_b, y_train_b)
val_pred_bias = lgb_bias.predict_proba(X_val_b)
val_logloss_bias = log_loss(y_val_b, val_pred_bias)
print(f"🎯 Validation LogLoss (Bias-aware LGBM + PCA): {val_logloss_bias:.5f}")


In [ ]:
# === In-depth Position Bias Analysis (Revised Version) ===
print("\n--- Executing: In-depth Position Bias Analysis ---")

# Randomly sample from training set and reset index
subset = train.sample(1000, random_state=42).reset_index(drop=True)
subset_swapped = subset.copy()

# Swap response_a and response_b
subset_swapped["response_a"], subset_swapped["response_b"] = (
    subset["response_b"], subset["response_a"]
)

# Generate input texts (Prompt + A + B concatenation)
subset_texts = (subset["prompt"] + " " + subset["response_a"] + " " + subset["response_b"]).tolist()
subset_texts_swapped = (subset_swapped["prompt"] + " " + subset_swapped["response_a"] + " " + subset_swapped["response_b"]).tolist()

# Generate embeddings and predictions
subset_emb = model.encode(subset_texts, show_progress_bar=False)
subset_emb_swapped = model.encode(subset_texts_swapped, show_progress_bar=False)

pred_orig = clf_emb.predict_proba(subset_emb)
pred_swap = clf_emb.predict_proba(subset_emb_swapped)

# Calculate "prediction flip rate" (should be high if model truly focuses on content)
flip_rate = np.mean(np.argmax(pred_orig, axis=1) != np.argmax(pred_swap, axis=1))
print(f"🔄 Model prediction flip rate (after A/B swap): {flip_rate:.3f}")

# Compare average prediction probabilities
avg_conf_diff = np.mean(np.abs(pred_orig - pred_swap))
print(f"📊 Average probability change magnitude: {avg_conf_diff:.4f}")


### 5.3 Lightweight Fine-tuning and Probability Calibration

**Objective:**
Attempt to adapt the Transformer model for downstream tasks using the lightweight parameter-efficient tuning (PEFT/LoRA) method, and enhance the reliability of prediction confidence through calibration. 

#### (1) LoRA Lightweight Fine-Tuning

- **Model:** `microsoft/deberta-v3-small`
- **Method:** Utilize the LoRA configuration of PEFT with `r=8`, `lora_alpha=16`, and `target_modules=["query_proj", "value_proj"]`.
- **Training:** Use the concatenated input of prompt + responseA/B, with the parameters of the LoRA layer trainable and the rest of the weights frozen. 
- **Advantages:** Significantly reduces memory consumption, only requiring a few hundred MB of memory to complete the fine-tuning.
- **Effect:** The LogLoss on the validation set is approximately 1.07, showing a slight improvement compared to the Embedding model.

> Note: In this cell, we only preliminarily apply the LoRA fine-tuning method and do not attempt to concatenate the three dimension feature vector `["len_diff", "len_ratio", "lexical_diff"]` as completed in the previous steps with the 384 dimension vector gotten from LLM; this attempt will be completed in the final model design after error analysis.

In [ ]:
# === Step 5.3: LoRA Fine-tuning (DeBERTa-small, Memory-Optimized) ===
print("\n--- Executing Step 5.3: LoRA Fine-tuning (Memory Optimized) ---")

import os
os.environ["WANDB_MODE"] = "disabled"  # Disable W&B

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
import torch
import pandas as pd
import shutil
from sklearn.metrics import accuracy_score

# 1️⃣ Kaggle Input Model Path
input_model_path = "/kaggle/input/deberta-v3-small/deberta-v3-small"
local_model_path = "./deberta-small-local"
if not os.path.exists(local_model_path):
    shutil.copytree(input_model_path, local_model_path)

# 2️⃣ Load Local Model and Tokenizer
tokenizer = AutoTokenizer.from_pretrained(local_model_path, local_files_only=True)
base_model = AutoModelForSequenceClassification.from_pretrained(
    local_model_path, num_labels=3, local_files_only=True
)

# 3️⃣ Configure LoRA
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(base_model, peft_config)

# 4️⃣ Data Processing: Concatenate prompt and options
def preprocess_function(examples):
    texts = [
        f"Question: {p} [SEP] A: {a} [SEP] B: {b}" 
        for p, a, b in zip(examples["prompt"], examples["response_a"], examples["response_b"])
    ]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=256)

train_texts_ft = train[:-2000]
y_train_ft = y[:-2000]
val_texts = train[-2000:]
y_val_ft = y[-2000:]

train_dataset = Dataset.from_dict({
    "prompt": train_texts_ft["prompt"],
    "response_a": train_texts_ft["response_a"],
    "response_b": train_texts_ft["response_b"],
    "label": y_train_ft
})
val_dataset = Dataset.from_dict({
    "prompt": val_texts["prompt"],
    "response_a": val_texts["response_a"],
    "response_b": val_texts["response_b"],
    "label": y_val_ft
})

# Parallel tokenization for speed
tokenized_train = train_dataset.map(preprocess_function, batched=True, num_proc=2)
tokenized_val = val_dataset.map(preprocess_function, batched=True, num_proc=2)

# 5️⃣ Evaluation Function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

# 6️⃣ Training Configuration (Memory Optimized)
training_args = TrainingArguments(
    output_dir="./ft_results",
    per_device_train_batch_size=8,          # Reduced batch size
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,         # 2-step accumulation, equivalent to batch_size=16
    num_train_epochs=3,
    learning_rate=3e-4,
    logging_steps=50,
    fp16=True,                              # Mixed precision training
    report_to=[]                            # Disable W&B
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 7️⃣ Start Fine-tuning
print("⏳ Starting LoRA fine-tuning...")
trainer.train()
print("✅ Fine-tuning completed.")

# 8️⃣ Test Set Prediction
test_texts = [
    f"Question: {p} [SEP] A: {a} [SEP] B: {b}" 
    for p, a, b in zip(test["prompt"], test["response_a"], test["response_b"])
]
test_dataset = Dataset.from_dict({"text": test_texts})
tokenized_test = test_dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=256), batched=True, num_proc=2)

pred_logits = trainer.predict(tokenized_test).predictions
pred_probs = torch.softmax(torch.tensor(pred_logits), dim=-1).numpy()

submission_ft = pd.DataFrame(pred_probs, columns=sample.columns[1:])
submission_ft.insert(0, "id", sample["id"])
submission_ft.to_csv("submission_finetuned.csv", index=False)
print("✅ Fine-tuned model results saved as submission_finetuned.csv")


#### (2) Temperature Scaling Probability Calibration
- **Principle:** Divide the logits by temperature T (>0) to minimize the validation log loss and determine the optimal T.
- **Implementation:** Use `scipy.optimize.minimize` to search for the optimal temperature within the range [0.5, 5.0].
- **Result:** The best T ≈ 1.64. After calibration, the model's output probability distribution becomes smoother and more in line with the actual confidence level.
- **Output:** Generate `submission_calibrated.csv` as the final calibrated result.

In [ ]:
# === Calibration (Temperature Scaling) ===
print("\n--- Executing: Probability Calibration (Temperature Scaling) ---")

from sklearn.metrics import log_loss
from scipy.optimize import minimize
import numpy as np

# Use validation set logits
val_logits = trainer.predict(tokenized_val).predictions
val_probs = torch.softmax(torch.tensor(val_logits), dim=-1).numpy()

def temperature_scale(logits, T):
    logits_T = logits / T
    exp_T = np.exp(logits_T - np.max(logits_T, axis=1, keepdims=True))
    return exp_T / np.sum(exp_T, axis=1, keepdims=True)

def loss_fn(T):
    probs_T = temperature_scale(val_logits, T)
    return log_loss(y_val_ft, probs_T)

# Optimize temperature parameter T
res = minimize(loss_fn, x0=[1.0], bounds=[(0.5, 5.0)], method="L-BFGS-B")
T_opt = res.x[0]
print(f"📏 Optimal temperature parameter T = {T_opt:.3f}")

# Apply to test set predictions
calibrated_probs = temperature_scale(pred_logits, T_opt)

submission_calibrated = pd.DataFrame(calibrated_probs, columns=sample.columns[1:])
submission_calibrated.insert(0, "id", sample["id"])
submission_calibrated.to_csv("submission_calibrated.csv", index=False)
submission_calibrated.to_csv("submission.csv", index=False)
print("✅ Calibrated results saved as submission_calibrated.csv")


 #### ✅ Summary
| Model Type | Method | Validation LogLoss |
|-----------|------|---------------|
| MiniLM Embedding + LR | Baseline embedding model | 1.0849 |
| E5 + LightGBM | Single model extension | 1.0838 |
| MiniLM + E5 Ensemble | Model ensemble | 1.0767 |
| Bias-aware LGBM + PCA | Adding bias features | 1.0326 |
| DeBERTa-small + LoRA | Lightweight fine-tuning | ≈ 1.07 |
| LoRA + Temperature Scaling | Post-tuning calibration | ≈ 1.05 (final submission) |

> Through model extension, bias modeling, lightweight fine-tuning and calibration, the model achieved significant improvement over baseline on validation set LogLoss, with enhanced performance and reliability.

## 6. Error Analysis

## 7. Final Model

In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import gc
import joblib 
import lightgbm as lgb
from scipy.sparse import hstack
import warnings

# Feature Engineering
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Evaluation & Validation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

# Disable unnecessary warnings
warnings.filterwarnings('ignore')
os.environ["WANDB_MODE"] = "disabled"

# Memory cleanup function
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("🚀 (Clean 5-Fold CV, Model A Only) Pipeline Started!")

# --- 1. Competition Data Paths ---
COMP_DIR = "/kaggle/input/llm-classification-finetuning"
TRAIN_FILE = os.path.join(COMP_DIR, "train.csv")
TEST_FILE = os.path.join(COMP_DIR, "test.csv")
SAMPLE_FILE = os.path.join(COMP_DIR, "sample_submission.csv")

# --- 2. Public Model Paths ---
print("⏳ Defining public model paths...")
BASE_MINILM_PATH = "/kaggle/input/sentencetransformersallminilml6v2"
print(f"  ...MiniLM Path: {BASE_MINILM_PATH}")

print("✅ All paths defined.")
print(f"⏳ Loading train.csv and test.csv...")

In [ ]:
# === Cell 2: Feature Enginer ===
try:
    train_df = pd.read_csv(TRAIN_FILE)
    test_df = pd.read_csv(TEST_FILE)
    sample_df = pd.read_csv(SAMPLE_FILE) # sample_df will be used in final cell
    print(f"  Train: {train_df.shape}, Test: {test_df.shape}")
except FileNotFoundError as e:
    print(f"❌ Data loading failed! {e}")
    raise

# --- 2.1 Base Features (len, lexical) ---
print("⏳ Creating base features (len_diff, lexical_diff)...")
def create_base_features(df):
    df['text_a'] = df['prompt'] + " " + df['response_a']
    df['text_b'] = df['prompt'] + " " + df['response_b']
    df['combined_for_embedding'] = df['text_a'] + " [SEP] " + df['text_b']
    df["resp_a_len"] = df["response_a"].str.len()
    df["resp_b_len"] = df["response_b"].str.len()
    df["len_diff"] = df["resp_a_len"] - df["resp_b_len"]
    df["len_ratio"] = df["resp_a_len"] / (df["resp_b_len"] + 1e-6)
    df["lexical_a"] = df["response_a"].apply(lambda x: len(set(str(x).split())) / (len(str(x).split()) + 1e-6))
    df["lexical_b"] = df["response_b"].apply(lambda x: len(set(str(x).split())) / (len(str(x).split()) + 1e-6))
    df["lexical_diff"] = df["lexical_a"] - df["lexical_b"]
    return df

train_df = create_base_features(train_df)
test_df = create_base_features(test_df)
train_df['label'] = train_df[['winner_model_a', 'winner_model_b', 'winner_tie']].values.argmax(axis=1)
y_true_full = train_df['label'] # Prepare all labels (57k)

# --- 2.2 SBERT Embeddings (MiniLM) ---
print("⏳ Generating MiniLM embeddings (for A and similarity)...")
model_minilm = SentenceTransformer(BASE_MINILM_PATH, device='cuda') 
train_emb_minilm = model_minilm.encode(train_df['combined_for_embedding'].tolist(), show_progress_bar=True, batch_size=128)
test_emb_minilm = model_minilm.encode(test_df['combined_for_embedding'].tolist(), show_progress_bar=True, batch_size=128)

# --- 2.3 Similarity Features (from MiniLM) ---
print("⏳ Creating similarity features...")
resp_a_emb_train = model_minilm.encode(train_df['response_a'].tolist(), show_progress_bar=True, batch_size=128)
resp_b_emb_train = model_minilm.encode(train_df['response_b'].tolist(), show_progress_bar=True, batch_size=128)
train_df['cosine_similarity'] = np.array([cosine_similarity(resp_a_emb_train[i].reshape(1, -1), resp_b_emb_train[i].reshape(1, -1))[0][0] for i in range(len(resp_a_emb_train))])

resp_a_emb_test = model_minilm.encode(test_df['response_a'].tolist(), show_progress_bar=True, batch_size=128)
resp_b_emb_test = model_minilm.encode(test_df['response_b'].tolist(), show_progress_bar=True, batch_size=128)
test_df['cosine_similarity'] = np.array([cosine_similarity(resp_a_emb_test[i].reshape(1, -1), resp_b_emb_test[i].reshape(1, -1))[0][0] for i in range(len(resp_a_emb_test))])

del model_minilm, resp_a_emb_train, resp_b_emb_train, resp_a_emb_test, resp_b_emb_test
clear_memory()

# --- 2.4 N-gram Features (On-the-fly Training) ---
print("⏳ Training N-gram Vectorizer and creating features...")
corpus = pd.concat([train_df['response_a'], train_df['response_b']]).astype(str).unique()

vectorizer = CountVectorizer(
    max_features=2000,
    ngram_range=(1, 2), # Include 1-grams and 2-grams
    stop_words='english',
    dtype=np.float32 
)
print("  ...Fitting Vectorizer...")
vectorizer.fit(corpus)
del corpus
clear_memory()

print("  ...Transforming train/test sets...")
# [[[ Logic Fix ]]]: Correctly create difference features
train_ngram_a = vectorizer.transform(train_df['response_a'].astype(str))
train_ngram_b = vectorizer.transform(train_df['response_b'].astype(str))
train_ngram_diff = (train_ngram_a - train_ngram_b)

test_ngram_a = vectorizer.transform(test_df['response_a'].astype(str))
test_ngram_b = vectorizer.transform(test_df['response_b'].astype(str))
test_ngram_diff = (test_ngram_a - test_ngram_b)

del vectorizer, train_ngram_a, train_ngram_b, test_ngram_a, test_ngram_b
clear_memory()

# --- 2.5 [[[ NameError Fix ]]] ---
# Extract 4 bias features before deleting train_df/test_df
print("⏳ Extracting 4 bias features...")
all_4_features_train = train_df[["len_diff", "len_ratio", "lexical_diff", "cosine_similarity"]].fillna(0).values
all_4_features_test = test_df[["len_diff", "len_ratio", "lexical_diff", "cosine_similarity"]].fillna(0).values

# --- 2.6 Now safe to delete ---
del train_df, test_df
clear_memory()

print("✅ All feature engineering completed.")



In [ ]:
# === Cell 3: Prepare Final Feature Matrices ===
print("\n--- Preparing final train/test feature matrices ---")
# (All variables train_emb_minilm, all_4_features_train, train_ngram_diff etc. already in memory)

# --- Stack Package A (MiniLM + 4 Feat + Ngram) ---
print(f"  ...Stacking Package A (MiniLM + 4 Feat + Ngram, total {384 + 4 + 2000} features)")
X_A_full = hstack([train_emb_minilm, all_4_features_train, train_ngram_diff]).tocsr()
X_test_A_ngram = hstack([test_emb_minilm, all_4_features_test, test_ngram_diff]).tocsr()

print(f"✅ Feature matrices ready. Train: {X_A_full.shape}, Test: {X_test_A_ngram.shape}")

# --- Free Memory ---
del train_emb_minilm, test_emb_minilm, all_4_features_train, all_4_features_test, train_ngram_diff, test_ngram_diff
clear_memory()

# === Cell 4: Define 5-Fold Cross Validation ===
print("\n--- Defining 5-Fold CV ---")

N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# (y_true_full from Cell 2)

# Define LGBM Parameters
lgbm_params = {
    'n_estimators': 300,
    'learning_rate': 0.05,
    'num_leaves': 64,
    'random_state': 42,
    'device': 'gpu',
    'n_jobs': -1,
    'verbose': -1
}

# --- Initialize Storage ---
oof_preds_A = np.zeros((len(y_true_full), 3))
test_preds_A_list = []

print(f"✅ K-Fold (n_splits={N_SPLITS}) ready.")
